# Write Reyes probe recordings as NWB

* 40 seconds recording at 20 Khz.

TODO:
- Bundle into a NWB 1
    - isolated cell 3754013 probe-wired raw traces as a RecordingExtractor
    - isolated cell 3754013 spike train as a Ground truth SortingExtractor

- Bundle into a NWB 2
    - summed-cell probe-wired raw traces as a RecordingExtractor 2
    - all spike trains as a SortingExtractor 

## Setup

activate spikeinterface..


In [16]:
%load_ext autoreload
%autoreload 2

import spikeinterface as si
import pandas as pd
import numpy as np
import spikeinterface.extractors as se
import yaml
import os
import shutil 

# move to project path
with open("./proj_cfg.yml", "r", encoding="utf-8") as proj_cfg:
    PROJ_PATH = yaml.load(proj_cfg, Loader=yaml.FullLoader)["proj_path"]
os.chdir(PROJ_PATH)

# custom package
from src.nodes.utils import get_config
from src.nodes.prepro import preprocess

# SETUP PARAMETERS
data_conf, param_conf = get_config("silico_reyes", "2023_01_11").values()

SFREQ = param_conf["sampling_freq"]

# SETUP CONFIG
# all cells
RAW_PATH = data_conf["dataeng"]["campaign"]["output"]["trace_file_path"]
REC_PATH = data_conf["probe_wiring"]["full"]["output"]
GT_ALL_PATH = data_conf["ground_truth"]["full"]["output"]

# isolated cell
ISOLATED_CELL_RAW_PATH = data_conf["probe_wiring_isolated_cell"]["full"]["input"]
ISOLATED_CELL_REC_PATH = data_conf["probe_wiring_isolated_cell"]["full"]["output"]
GT_ISOLATED_CELL_PATH = data_conf["ground_truth_isolated_cell"]["full"]["output"]
#NWB_PATH = data_conf["nwb"]

job_dict = {"n_jobs": 1, "chunk_memory": None, "progress_bar": True} # butterworth


CELL = 3754013 # the most active pyramidal cell


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-11-21 12:36:10,458 - root - utils.py - get_config - INFO - Reading experiment config.
2024-11-21 12:36:10,486 - root - utils.py - get_config - INFO - Reading experiment config. - done


## 1. Cast summed trace and ground truth SortingExtractor as NWB

In [17]:
# paramters
SAVE = False

# RecordingExtractor ----------

# study raw isolated cell "3754013" extracellular trace
trace_df = pd.read_pickle(RAW_PATH)
print("print trace data: ")
display(trace_df.head(3))
print("trace data shape: ", trace_df.shape)

# RecordingExtractor with wired probe
Recording = si.load_extractor(REC_PATH)

# unit-test that raw all-cell summed traces were cast as a RecordingExtractor and wired
display(Recording)
assert (Recording.get_traces() == trace_df.values).all(), "should be the same"


# SortingExtractor ----------

if SAVE:
    preprocess.save_ground_truth(data_conf, param_conf)
SortingTrue = si.load_extractor(GT_ALL_PATH)
SortingTrue

print trace data: 


contact,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
time,,,,,,,,,,,,,,,,,,,,,
0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.05,0.854850,0.926285,0.969437,0.985312,0.981871,0.924142,0.859691,0.792676,0.708985,0.624970,...,1.922165,1.805422,1.650496,1.471562,1.278500,1.075797,0.874729,0.689928,0.525782,0.380212
0.10,1.717738,1.832260,1.917133,1.963162,1.974469,1.911653,1.805506,1.660403,1.467338,1.245444,...,2.702916,2.508831,2.246128,1.931927,1.584378,1.207509,0.826894,0.460708,0.119742,-0.189310


trace data shape:  (799900, 128)


BinaryFolderRecording: 128 channels - 20.0kHz - 1 segments - 799,900 samples - 39.99s 
                       float32 dtype - 390.58 MiB

NumpyFolderSorting: 1 units - 1 segments - 20.0kHz

## 2. Cast isolated cell recording as Recording and SortingExtractors

In [18]:
SAVE = True

# RecordingExtractor ----------

# study raw isolated cell "3754013" extracellular trace
cell_trace_df = pd.read_pickle(ISOLATED_CELL_RAW_PATH)
print("print trace data: ")
display(cell_trace_df.head(3))
print("trace data shape: ", cell_trace_df.shape)

# isolated cell
# read and cast raw trace as array (1 min/h recording)
trace = np.array(cell_trace_df)

# cast as RecordingExtractor
CellRecording = se.NumpyRecording(
    traces_list=[trace],
    sampling_frequency=SFREQ,
)

# wire probe
Probe = Recording.get_probe()
CellRecording = CellRecording.set_probe(Probe)

if SAVE:
    shutil.rmtree(ISOLATED_CELL_REC_PATH, ignore_errors=True)
    CellRecording.save(folder=ISOLATED_CELL_REC_PATH, format="binary", **job_dict)


# SortingExtractor ----------

# select unit
CellSortingTrue = SortingTrue.select_units([CELL])

# save
if SAVE:
    shutil.rmtree(GT_ISOLATED_CELL_PATH, ignore_errors=True)
    CellSortingTrue.save(folder=GT_ISOLATED_CELL_PATH, n_jobs=-1, total_memory="2G")

print trace data: 


contact,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
time,,,,,,,,,,,,,,,,,,,,,
0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
0.05,4.085209e-07,4.149662e-07,4.117478e-07,3.961939e-07,3.662417e-07,3.211508e-07,2.620635e-07,1.922216e-07,1.167273e-07,4.132322e-08,...,-4.675192e-08,-2.160839e-07,-3.772431e-07,-5.096762e-07,-6.008838e-07,-6.495970e-07,-6.612717e-07,-6.475194e-07,-6.108052e-07,-5.735339e-07
0.10,6.108059e-07,6.265054e-07,6.291554e-07,6.144989e-07,5.791312e-07,5.212871e-07,4.418467e-07,3.443373e-07,2.359972e-07,1.248999e-07,...,2.204069e-08,-2.121559e-07,-4.416914e-07,-6.371887e-07,-7.785264e-07,-8.622193e-07,-8.924056e-07,-8.860252e-07,-8.484813e-07,-8.055820e-07


trace data shape:  (799900, 128)
write_binary_recording with n_jobs = 1 and chunk_size = 20000


write_binary_recording:   0%|          | 0/40 [00:00<?, ?it/s]

## 3. TEST the NWBs

In [34]:
import spikeinterface.extractors as se

data_conf, param_conf = get_config("silico_reyes", "2023_01_11").values()
REC_PATH = data_conf["probe_wiring"]["full"]["output"]
GT_PATH = data_conf["ground_truth"]["full"]["output"]
NWB_PATH = data_conf["probe_wiring_nwb"]  # write path

# get NWB
# from sbatch cluster/dandi/write_nwb/raw/biophy_summed_reyes.sh
RecordingSum = se.NwbRecordingExtractor(NWB_PATH)

# unit-test NWB
assert (
    RecordingSum.get_traces() == Recording.get_traces()
).all(), "Should be identical"

2024-11-21 12:52:48,406 - root - utils.py - get_config - INFO - Reading experiment config.
2024-11-21 12:52:48,426 - root - utils.py - get_config - INFO - Reading experiment config. - done


In [33]:
REC_PATH = data_conf["probe_wiring_isolated_cell"]["full"]["output"]
GT_PATH = data_conf["ground_truth_isolated_cell"]["full"]["output"]
NWB_PATH = data_conf["probe_wiring_isolated_cell_nwb"]  # write path

# get NWB
# from sbatch cluster/dandi/write_nwb/raw/biophy_isolated_reyes.sh
RecordingIsol = se.NwbRecordingExtractor(NWB_PATH)

# unit-test NWB
assert (
    RecordingIsol.get_traces() == CellRecording.get_traces()
).all(), "Should be identical"

## 4. Upload to DANDI archive

sbatch cluster/dandi/upload/fitted/upload.sh